In [1]:
import ROOT
import math
from array import array

Welcome to JupyROOT 6.26/00


In [2]:
%%bash
pwd

/Users/akhanal/work/Expt_phys/simulation/xi_1820/test


In [3]:
ls -l *.root

-rw-r--r--  1 akhanal  staff       44490 Feb 13 17:08 basic.root
-rw-r--r--  1 akhanal  staff      890817 Feb 22 16:41 data_all_fde.root
-rw-r--r--  1 akhanal  staff     7487446 Feb 22 16:38 data_all_fte.root
-rw-r--r--  1 akhanal  staff     7391613 Feb 22 16:57 data_all_fte_ecornonly.root
-rw-r--r--  1 akhanal  staff     7233824 Feb 22 17:00 data_all_fte_nocorn.root
-rw-r--r--  1 akhanal  staff      243193 Feb 21 19:22 data_fall2018_in_fde.root
-rw-r--r--  1 akhanal  staff     3130991 Feb 22 18:21 data_fall2018_in_fte.root
-rw-r--r--  1 akhanal  staff     3090975 Feb 22 18:19 data_fall2018_in_fte_ecornonly.root
-rw-r--r--  1 akhanal  staff     3025107 Feb 22 18:17 data_fall2018_in_fte_nocorn.root
-rw-r--r--  1 akhanal  staff      422232 Feb 22 16:18 data_fall2018_out_fde.root
-rw-r--r--  1 akhanal  staff     1280945 Feb 22 18:11 data_fall2018_out_fte.root
-rw-r--r--  1 akhanal  staff     1264513 Feb 22 18:13 data_fall2018_out_fte_ecornonly.root
-rw-r--r--  1 akhanal  staff     1237557

In [4]:
%%bash
rootls -l data_fall2018_out_fde.root

TTree  Feb 22 16:18 2022 T;1 "data from ascii file" 


In [5]:
data_tree = "T"
gen_mctree = "T"
rec_mctree = "T"
data_file = "data_fall2018_out_fde.root"
gen_mcfile = "mc_gen_fall2018_out_fde.root"
rec_mcfile = "mc_rec_fall2018_out_fde.root"
df = ROOT.RDataFrame(data_tree, data_file)
genmcdf = ROOT.RDataFrame(gen_mctree, gen_mcfile)
recmcdf = ROOT.RDataFrame(rec_mctree, rec_mcfile)
print(f"Columns in the dataset: {df.GetColumnNames()}")
#print(f"Columns in the dataset: {genmcdf.GetColumnNames()}")
#print(f"Columns in the dataset: {recmcdf.GetColumnNames()}")

Columns in the dataset: { "W", "beam_e", "e_e", "e_p", "e_phi", "e_px", "e_py", "e_pz", "e_theta", "e_vx", "e_vy", "e_vz", "evnum", "fkp_e", "fkp_p", "fkp_phi", "fkp_px", "fkp_py", "fkp_pz", "fkp_theta", "fkp_vx", "fkp_vy", "fkp_vz", "km_e", "km_p", "km_phi", "km_px", "km_py", "km_pz", "km_theta", "km_vx", "km_vy", "km_vz", "m_fkpkm", "m_fkpskp", "m_fkpskpkm", "m_skpkm", "mm_e", "mm_fkp", "mm_fkpaspipskpaspipkm", "mm_fkpaspipskpaspipkmaspim", "mm_fkpaspipskpkm", "mm_fkpaspipskpkmaspim", "mm_fkpkm", "mm_fkpskp", "mm_fkpskpaspipkm", "mm_fkpskpaspipkmaspim", "mm_fkpskpkm", "mm_fkpskpkmaspim", "mm_km", "mm_skp", "mm_skpkm", "nu", "q2", "runnum", "skp_e", "skp_p", "skp_phi", "skp_px", "skp_py", "skp_pz", "skp_theta", "skp_vx", "skp_vy", "skp_vz", "x" }


In [6]:
%%cpp 
double background(Double_t *x, Double_t *par) {
  return par[0] + par[1]*x[0];
}

In [7]:
%%cpp
Double_t p2background(Double_t *x, Double_t *par) {
  //return par[0] + par[1]*x[0] + par[2]*x[0]*x[0]+ par[3]*x[0]*x[0]*x[0];
  return par[0] + par[1]*x[0] + par[2]*x[0]*x[0];
}

In [8]:
%%cpp
Double_t p3background(Double_t *x, Double_t *par) {
  return par[0] + par[1]*x[0] + par[2]*x[0]*x[0]+ par[3]*x[0]*x[0]*x[0];
  
}

In [9]:
%%cpp 
double backgroundPoly(Double_t *x) {
  return 2*x[0]*x[0]-1;
}

In [10]:
%%cpp
Double_t Gaussian(Double_t *x, Double_t *par)
{
    Double_t arg = 0;
    if (par[2]) arg = (x[0] - par[1])/par[2];
    Double_t sig = par[0]*TMath::Exp(-0.5*arg*arg);
    return sig;
}

In [11]:
%%cpp
Double_t GaussianPeak(Double_t *x, Double_t *par)
{
    Double_t arg = 0;
    if (par[2]) arg = (x[0] - par[1])/par[2];
    Double_t sig = par[0]*TMath::Exp(-0.5*arg*arg);
    return sig;
}

In [12]:
%%cpp
double p2fitDoubleGaus(double *x, double *par) {
  return GaussianPeak(x, par) + Gaussian(x, &par[3]) + p2background(x, &par[6]);
}

In [13]:
%%cpp
double fitDoubleGaus(double *x, double *par) {
  return GaussianPeak(x, par) + Gaussian(x, &par[3]) + background(x, &par[6]);
}

In [14]:
%%cpp
double fitGaus(double *x, double *par) {
  return GaussianPeak(x, par) + p2background(x, &par[3]);
}

In [15]:
%%cpp
double fitp3Gaus(double *x, double *par) {
  return GaussianPeak(x, par) + p3background(x, &par[3]);
    //+ par[3]+par[4]*x[0]+par[5]*x[0]*x[0]+par[6]*x[0]*x[0]*x[0];
}

In [16]:
%%cpp
Double_t lorentzianPeak(Double_t *x, Double_t *par) {
    return (0.5*par[0]*par[2]/TMath::Pi()) /
    TMath::Max( 1.e-10,
               (x[0]-par[1])*(x[0]-par[1]) + .25*par[2]*par[2]
               );
}

In [17]:
%%cpp
Double_t fitLorentzianPeak(Double_t *x, Double_t *par) {
    return lorentzianPeak(x, par) + p2background(x, &par[3]);
    //return lorentzianPeak(x, par) + backgroundPoly(x);
}

In [18]:
%%cpp
double fitBW(double *x, double *par) {
  return lorentzianPeak(x, par) + background(x, &par[3]);
}

In [19]:
%%cpp
float low_3sigma(TF1 *fcn){
    return fcn->GetParameter(1)-3*fcn->GetParameter(2);
}

In [20]:
%%cpp
float high_3sigma(TF1 *fcn){
    return fcn->GetParameter(4)+3*fcn->GetParameter(5);
}

In [21]:
%%cpp
bool sigmaCut(TF1 *fcn){
    if(fcn->GetParameter(1)-3*fcn->GetParameter(2) && fcn->GetParameter(4)+3*fcn->GetParameter(5)){
        return true;
    }
    else {
        return false;
    }  
}

In [22]:
#h1 = ROOT.RDataFrame("T","../simu_prod/xi_mc_fall2018out_fde/plot/rec_xi_mc_smeared0p1p_fall2018out_fde.root").Histo1D(("mc-rec", "mc-rec", 25, 0.9, 1.4), "mm_fkpskpkm")
#h1 = ROOT.RDataFrame("T","../simu_prod/xi_mc_fall2018out_fde/plot/rec_xi_mc_smeared0p2p_fall2018out_fde.root").Histo1D(("mc-rec", "mc-rec", 25, 0.9, 1.4), "mm_fkpskpkm")
#h1 = ROOT.RDataFrame("T","../simu_prod/xi_mc_fall2018out_fde/plot/rec_xi_mc_smeared0p3p_fall2018out_fde.root").Histo1D(("mc-rec", "mc-rec", 25, 0.9, 1.4), "mm_fkpskpkm")
#h1 = ROOT.RDataFrame("T","../simu_prod/xi_mc_fall2018out_fde/plot/rec_xi_mc_smeared0p35p_fall2018out_fde.root").Histo1D(("mc-rec", "mc-rec", 25, 0.9, 1.4), "mm_fkpskpkm")
#h1 = ROOT.RDataFrame("T","../simu_prod/xi_mc_fall2018out_fde/plot/rec_xi_mc_smeared0p4p_fall2018out_fde.root").Histo1D(("mc-rec", "mc-rec", 25, 0.9, 1.4), "mm_fkpskpkm")
#h1 = ROOT.RDataFrame("T","../simu_prod/xi_mc_fall2018out_fde/plot/rec_xi_mc_smeared0p5p_fall2018out_fde.root").Histo1D(("mc-rec", "mc-rec", 25, 0.9, 1.4), "mm_fkpskpkm")
#h1 = ROOT.RDataFrame("T","../simu_prod/xi_mc_fall2018out_fde/plot/rec_xi_mc_smeared0p6p_fall2018out_fde.root").Histo1D(("mc-rec", "mc-rec", 25, 0.9, 1.4), "mm_fkpskpkm")
#h1 = ROOT.RDataFrame("T","../simu_prod/xi_mc_fall2018out_fde/plot/rec_xi_mc_smeared0p7p_fall2018out_fde.root").Histo1D(("mc-rec", "mc-rec", 25, 0.9, 1.4), "mm_fkpskpkm")
#h1 = ROOT.RDataFrame("T","../simu_prod/xi_mc_fall2018out_fde/plot/rec_xi_mc_smeared0p8p_fall2018out_fde.root").Histo1D(("mc-rec", "mc-rec", 25, 0.9, 1.4), "mm_fkpskpkm")
#h1 = ROOT.RDataFrame("T","../simu_prod/xi_mc_fall2018out_fde/plot/rec_xi_mc_smeared1p_fall2018out_fde.root").Histo1D(("mc-rec", "mc-rec", 25, 0.9, 1.4), "mm_fkpskpkm")
#h1 = ROOT.RDataFrame("T","../simu_prod/xi_mc_fall2018out_fde/plot/rec_xi_mc_smeared1p2p_fall2018out_fde.root").Histo1D(("mc-rec", "mc-rec", 25, 0.9, 1.4), "mm_fkpskpkm")
h1 = ROOT.RDataFrame("T","../simu_prod/xi_mc_fall2018out_fde/plot/rec_xi_mc_smeared1p4p_fall2018out_fde.root").Histo1D(("mc-rec", "mc-rec", 25, 0.9, 1.4), "mm_fkpskpkm")
#h1 = ROOT.RDataFrame("T","data_fall2018_out_fde.root").Histo1D(("data","data", 50, 0.9, 1.4), "mm_fkpskpkm") #100, 0.0, 2.4 #25, 0.5, 2.0
#h1 = ROOT.RDataFrame("T","data_fall2018_in_fde.root").Histo1D(("data","data", 50, 0.9, 1.4), "mm_fkpskpkm")
#100, 0.5, 2.0 #50, 0.5, 2.0
h1.SetTitle("MM(eK^{+}K^{+}K^{-})")
#h1.SetTitleSize(0.06, "x")
h1.GetXaxis().SetTitle("MM(eK^{+}K^{+}K^{-}) [GeV/c^{2}]")
h1.GetYaxis().SetTitle("Counts/10 MeV/c^{2}")
h2 = ROOT.RDataFrame("T","../simu_prod/xi_mc_fall2018out_fde/plot/rec_xi_mc_smeared0p35p_fall2018out_fde.root").Histo1D(("data", "data", 50, 1.5, 3.5), "mm_fkpskp")#50, 1.5, 3.5
h2.GetXaxis().SetTitle("MM(eK^{+}K^{+}) [GeV/c^{2}]")
h2.GetYaxis().SetTitle("Counts/40 MeV/c^{2}")

h1.GetXaxis().SetTitleSize(0.05)
h1.GetYaxis().SetTitleSize(0.05)
h1.GetXaxis().SetTitleOffset(0.9)
#h1.GetYaxis().SetTitleOffset(0.9)
h2.GetXaxis().SetTitleSize(0.05)
h2.GetYaxis().SetTitleSize(0.05)
h2.GetXaxis().SetTitleOffset(0.9)
#h2.GetYaxis().SetTitleOffset(0.9)

%jsroot on
c = ROOT.TCanvas()
c.Divide(2,2)
c.cd(1)
ROOT.gStyle.SetOptStat(00000)
h1.Draw("E")
h1.SetLineColor(ROOT.kRed)
h1.SetMarkerStyle(21)
h1.SetMarkerSize(0.8)
#h1.SetStats(0)

# fit resonance
#h1_fitFcn = ROOT.TF1('h1_fitFcn', ROOT.fitGaus, 0.0, 2.4, 5)
#fitFcn = ROOT.TF1("fitFcn", ROOT.p2fitDoubleGaus, 0.5, 2.0, 9)
fitFcn = ROOT.TF1("fitFcn", ROOT.fitDoubleGaus, 0.5, 2.0, 8)
fitFcn.SetLineColor(ROOT.kMagenta)#kMagenta#kBlue
#h1_fitFcn = ROOT.TF1('h1_fitFcn', ROOT.fitBW, 0.9, 1.4, 5)
#h1_fitFcn.SetLineWidth(3)
#"""
fitFcn.SetParameter(0, h1.GetBinContent(h1.GetMaximumBin())) #peak h1.GetBinContent(h1.GetMaximumBin())
#fitFcn.SetParLimits(0, h1.GetBinContent(h1.GetMaximumBin())-6, h1.GetBinContent(h1.GetMaximumBin())+5)
fitFcn.SetParameter(1, 1.115683) #mean h1.GetBinCenter(h1.GetMaximumBin())#1.115683
fitFcn.SetParLimits(1, 1.09, 1.13)
fitFcn.SetParameter(2, 0.04) #sigma
fitFcn.SetParLimits(2, 0.01, 0.06)
fitFcn.SetParameter(3, h1.GetBinContent(h1.GetMaximumBin())) #peak
#fitFcn.SetParLimits(3, h1.GetBinContent(h1.GetMaximumBin())-21, h1.GetBinContent(h1.GetMaximumBin())-15)
fitFcn.SetParameter(4, 1.192642) #mean#1.192642
fitFcn.SetParLimits(4, 1.18, 1.2)
fitFcn.SetParameter(5, 0.04) #sigma
fitFcn.SetParLimits(5, 0.01, 0.06)
fitFcn.SetParameter(6, 0.2) #p0
#fitFcn.SetParLimits(6, -2, 0.3)
fitFcn.SetParameter(7, 2.0) #p1(7, 0.6) #p1
#fitFcn.SetParLimits(7, 0.1, 5)
#fitFcn.SetParameter(8, 0) #p2
#fitFcn.SetParLimits(8, -0.2, -10)
#h1_fitFcn.SetParameter(8, -3) #p3
#h1_fitFcn.SetParameter(9, 0.9) #p4
#"""

"""
#BW fit
h1_fitFcn.SetParameter(0, 1.1)
h1_fitFcn.SetParameter(1, 42)
h1_fitFcn.SetParLimits(1, 40, 50)
h1_fitFcn.SetParameter(2, 0.02)
h1_fitFcn.SetParameter(3, 0.06)
h1_fitFcn.SetParameter(4, 0.6)
#"""

#h1.Fit("fitFcn", "EV","",0.9,1.4) # Global Fit
h1.Fit("fitFcn", "S","",0.9,1.4)
#h1.Fit("h1_fitFcn","V+","ep");

#"""
xsq = fitFcn.GetChisquare()
ndf = fitFcn.GetNDF()

chi2perndf = xsq/ndf

#improve the picture:
backFcn = ROOT.TF1("backFcn", ROOT.background, 0.9, 1.4, 2)
backFcn.SetParameters(fitFcn.GetParameter(6), fitFcn.GetParameter(7))
backFcn.SetLineColor(ROOT.kRed);
lamSignalFcn = ROOT.TF1("lamSignalFcn", ROOT.GaussianPeak, 0.9, 1.4, 3)
lamSignalFcn.SetParameters(fitFcn.GetParameter(0), fitFcn.GetParameter(1), fitFcn.GetParameter(2))
lamSignalFcn.SetLineColor(ROOT.kBlue)#ROOT.kGreen
sigSignalFcn = ROOT.TF1("sigSignalFcn", ROOT.Gaussian, 0.9, 1.4, 3)
sigSignalFcn.SetParameters(fitFcn.GetParameter(3), fitFcn.GetParameter(4), fitFcn.GetParameter(5))
sigSignalFcn.SetLineColor(ROOT.kGreen)#ROOT.kBlue
#signalFcn.SetNpx(500)
#h1_fitFcn.Draw("same")
backFcn.Draw("same")
lamSignalFcn.Draw("same")
sigSignalFcn.Draw("same")

#Intergal from (3sigma)signal and background function
factor = 3

Intg1 = lamSignalFcn.Integral(fitFcn.GetParameter(1)-factor*fitFcn.GetParameter(2), fitFcn.GetParameter(1)+factor*fitFcn.GetParameter(2))
Intg2 = sigSignalFcn.Integral(fitFcn.GetParameter(4)-factor*fitFcn.GetParameter(5), fitFcn.GetParameter(4)+factor*fitFcn.GetParameter(5))
Intgb = backFcn.Integral(fitFcn.GetParameter(1)-factor*fitFcn.GetParameter(2), fitFcn.GetParameter(4)+factor*fitFcn.GetParameter(5))
#Yield per bin width
binw = h1.GetBinWidth(1)
Yield1 = Intg1/(binw)
Yield2 = Intg2/(binw)
bg = Intgb/(binw)

#bg
factorsb1 = 6
factorsb2 = 9
Intgb_sb1 = backFcn.Integral(fitFcn.GetParameter(1)-6*fitFcn.GetParameter(2), fitFcn.GetParameter(1)-factor*fitFcn.GetParameter(2))
bg_sb1 = Intgb_sb1/(binw)

Intgb_sb2 = backFcn.Integral(fitFcn.GetParameter(4)+factor*fitFcn.GetParameter(5), fitFcn.GetParameter(4)+9*fitFcn.GetParameter(5))
bg_sb2 = Intgb_sb2/(binw)

bg_sb = bg_sb1 + bg_sb2

print(f"Lambda Yield {Yield1:f}, Sigma Yield {Yield2:f}, Background {bg:f}, Background_sb1 {bg_sb1:f}, Background_sb2 {bg_sb2:f}, Background_sb {bg_sb:f}" )
#"""

#"""
legend = ROOT.TLegend(0.9,0.8,0.7,0.9)#legend = ROOT.TLegend()
legend.SetTextFont(72)
legend.SetTextSize(0.04)
legend.SetHeader("data_fall2018_out_fde","C") # option "C" allows to center the header
#legend.SetHeader("data_fall2018_in_fde","C") # option "C" allows to center the header
#legend.SetHeader("data_fall2018_out_fde","C") # option "C" allows to center the header
legend.AddEntry("data","Data","lpe")
legend.AddEntry("backFcn", "Backgroung Fit","l")
legend.AddEntry("lamSignalFcn", "Lambda Fit","l")
legend.AddEntry("sigSignalFcn", "Sigma Fit","l")
legend.AddEntry("fitFcn","Global Fit","l") #legend->AddEntry("f1","Function abs(#frac{sin(x)}{x})","l");
#legend.AddEntry("allcorn","all corn","l"); #legend->AddEntry("gr","Graph with error bars","lep");
#legend.Draw()
#"""

startx=h1.GetXaxis().GetXmin()+0.05*(h1.GetXaxis().GetXmax()-h1.GetXaxis().GetXmin())
starty=1.0*h1.GetMaximum()
starty1=0.9*h1.GetMaximum()
starty30=0.8*h1.GetMaximum()
starty0=0.5*h1.GetMaximum()
starty2=0.7*h1.GetMaximum()
starty3=0.6*h1.GetMaximum()
starty10=1.1*h1.GetMaximum()

summ= ROOT.TLatex(startx, starty, f"#mu1:{fitFcn.GetParameter(1):5.4f}#pm{fitFcn.GetParError(1):5.4f}")
sum1= ROOT.TLatex(startx, starty1, f"#sigma1:{fitFcn.GetParameter(2):5.4f}#pm{fitFcn.GetParError(2):5.4f}")
sum10= ROOT.TLatex(startx, starty10, f"Yield1:{Yield1:.0f}")
sum2= ROOT.TLatex(startx, starty2, f"mu2:{fitFcn.GetParameter(4):5.4f}#pm{fitFcn.GetParError(4):5.4f}")
sum3= ROOT.TLatex(startx, starty3, f"#sigma2:{fitFcn.GetParameter(5):5.4f}#pm{fitFcn.GetParError(5):5.4f}")
sum30= ROOT.TLatex(startx, starty30, f"Yield2:{Yield2:.0f}")
sum4= ROOT.TLatex(startx, starty0, f"#chi2/Ndf:{chi2perndf:5.2f}")
#"""

summ.SetTextSize(0.05)#0.065
summ.SetTextColor(ROOT.kBlue)
summ.Draw("same")
#"""
sum1.SetTextSize(0.05)
sum1.SetTextColor(ROOT.kBlue)
sum1.Draw("same")
sum10.SetTextSize(0.05)
sum10.SetTextColor(ROOT.kBlue)
sum10.Draw("same")
sum2.SetTextSize(0.05)
sum2.SetTextColor(ROOT.kGreen)
sum2.Draw("same")
sum3.SetTextSize(0.05)
sum3.SetTextColor(ROOT.kGreen)
sum3.Draw("same")
sum30.SetTextSize(0.05)
sum30.SetTextColor(ROOT.kGreen)
sum30.Draw("same")
sum4.SetTextSize(0.05)
sum4.SetTextColor(ROOT.kMagenta)
sum4.Draw("same")


c.cd(2)
h2.Draw()
h2.SetTitle("MM(eK^{+}K^{+})")
#low_3sigma = fitFcn.GetParameter(1)-factor*fitFcn.GetParameter(2)
#high_3sigma = fitFcn.GetParameter(4)+factor*fitFcn.GetParameter(5)
#cut = low_3sigma < mm_fkpskpkm
#df_3sigma = ROOT.RDataFrame("T","data_spring2019_in_fde.root").Define("im", ROOT.sigmaCut(fitFcn))
h2_im_kmSignal =  ROOT.RDataFrame("T","../simu_prod/xi_mc_fall2018out_fde/plot/rec_xi_mc_smeared0p35p_fall2018out_fde.root").Filter("1.0675<=mm_fkpskpkm && mm_fkpskpkm<=1.2481").Histo1D(("data-im", "data-im", 25, 1.5, 2.5), "mm_fkpskp")#50, 1.5, 3.5
h2_im_kmSignal.SetLineColor(ROOT.kRed) #(muLam-3sig) = 1.0675, (muLam-6sig) = 1.0054, (muLam-9sig) = 0.9433, (muSig+3sig) = 1.2481, (muSig+6sig) = 1.2985, (muSig+9sig) = 1.3489
h2_im_kmSignal.SetFillColor(ROOT.kRed) #mulam-6sig = 1.0054, (muSig+9sig) = 1.3489
h2_im_kmSignal.SetFillStyle(4050)
h2_im_kmSignal.GetXaxis().SetTitle("MM(eK^{+}K^{+}) [GeV/c^{2}]")
h2_im_kmSignal.GetYaxis().SetTitle("Counts/40 MeV/c^{2}")
h2_im_kmSignal.Sumw2()
h2_im_kmSignal.GetXaxis().SetTitleSize(0.05)
h2_im_kmSignal.GetYaxis().SetTitleSize(0.05)
h2_im_kmSignal.GetXaxis().SetTitleOffset(0.9)
#h2_im_kmSignal.GetYaxis().SetTitleOffset(0.9)
#c.cd(3)
h2_im_kmSignal.Draw("same")

#Filter("(0.9979<=mm_fkpskpkm && mm_fkpskpkm<1.0618) || (1.2671<mm_fkpskpkm && mm_fkpskpkm<=1.3424)") 3-6sigma sideband
#Filter("(0.934<=mm_fkpskpkm && mm_fkpskpkm<1.0618) || (1.2671<mm_fkpskpkm && mm_fkpskpkm<=1.4177)") 3-9sigma sideband
#h2_im_sideband1 =  ROOT.RDataFrame("T","data_fall2018_out_fde.root").Filter("(1.0054<=mm_fkpskpkm && mm_fkpskpkm<1.0675) || (1.2481<mm_fkpskpkm && mm_fkpskpkm<=1.2985)").Histo1D(("data-sb1", "data-sb1", 50, 1.5, 3.5), "mm_fkpskp")
h2_im_sideband1 =  ROOT.RDataFrame("T","../simu_prod/xi_mc_fall2018out_fde/plot/rec_xi_mc_smeared0p35p_fall2018out_fde.root").Filter("(1.0054<=mm_fkpskpkm && mm_fkpskpkm<1.0675) || (1.2481<mm_fkpskpkm && mm_fkpskpkm<=1.3489)").Histo1D(("data-sb1", "data-sb1", 25, 1.5, 2.5), "mm_fkpskp")
h2_im_sideband1.SetLineColor(ROOT.kGreen)
h2_im_sideband1.SetFillColor(ROOT.kGreen)
h2_im_sideband1.SetFillStyle(4050)
h2_im_sideband1.Sumw2()
h2_im_sideband1.GetXaxis().SetTitleSize(0.05)
h2_im_sideband1.GetYaxis().SetTitleSize(0.05)
h2_im_sideband1.GetXaxis().SetTitleOffset(0.9)
#h2_im_sideband1.GetYaxis().SetTitleOffset(0.9)
h2_im_sideband1.Draw("same")
#h2_im_sideband1.SetMarkerStyle(21)
#h2_im_sideband1.SetMarkerSize(0.8)



h2_im_sideband2 =  ROOT.RDataFrame("T","../simu_prod/xi_mc_fall2018out_fde/plot/rec_xi_mc_smeared0p35p_fall2018out_fde.root").Filter("(0.9433<=mm_fkpskpkm && mm_fkpskpkm<1.0675) || (1.2481<mm_fkpskpkm && mm_fkpskpkm<=1.3489)").Histo1D(("data-sb2", "data-sb2", 25, 1.5, 2.5), "mm_fkpskp")
h2_im_sideband2.SetLineColor(ROOT.kBlue)
h2_im_sideband2.SetFillColor(ROOT.kBlue)
h2_im_sideband2.SetFillStyle(4050)
h2_im_sideband2.Sumw2()
#h2_im_sideband2.Draw("same")
c.cd(3)
h_im1 = h2_im_kmSignal.Clone()
h_im1.SetTitle("MM(eK^{+}K^{+})")
h_im1.GetXaxis().SetTitle("MM(eK^{+}K^{+}) [GeV/c^{2}]")
h_im1.GetYaxis().SetTitle("Counts/40 MeV/c^{2}")
h_im1.Add(h2_im_sideband1.GetPtr(),-1)
h_im1.Sumw2()
h_im1.Draw()
c.cd(4)
h_im2 = h2_im_kmSignal.Clone()
h_im2.SetTitle("MM(eK^{+}K^{+})")
h_im2.GetXaxis().SetTitle("MM(eK^{+}K^{+}) [GeV/c^{2}]")
h_im2.GetYaxis().SetTitle("Counts/40 MeV/c^{2}")
h_im2.Add(h2_im_sideband2.GetPtr(),-1)
h_im2.Sumw2()
#h_im2.Draw("E")
c.Draw()
#"""

Lambda Yield 52746.311764, Sigma Yield 20173.889993, Background 3935.193560, Background_sb1 1042.959246, Background_sb2 4803.986734, Background_sb 5846.945980
 FCN=91.0607 FROM MIGRAD    STATUS=CONVERGED    1098 CALLS        1099 TOTAL
                     EDM=1.2455e-08    STRATEGY= 1  ERROR MATRIX UNCERTAINTY   2.0 per cent
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           7.46059e+03   7.36352e+01   2.06955e-02   1.98940e-06
   2  p1           1.12746e+00   1.50473e-03   9.04105e-05  -3.57836e-03
   3  p2           5.65631e-02   8.18575e-04   4.40742e-05   1.44900e-03
   4  p3           2.76730e+03   1.99480e+02  -1.11491e-01  -1.81498e-06
   5  p4           1.20000e+00   1.67431e-02  -1.44657e-03** at limit **
   6  p5           5.83241e-02   1.46534e-03  -7.31652e-05   4.01573e-04
   7  p6          -7.25839e+01   3.23026e+01   4.82375e-03  -7.71801e-06
   8  p7         

Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created
Warning in <TH1D::Sumw2>: Sum of squares of weights structure already created


**DATA VS MC(smeared)**

In [62]:
p_smear = array('d', [0.1, 0.2, 0.3, 0.35, 0.4, 0.5, 0.6, 0.7, 0.8, 1.0, 1.2, 1.4])
err_p_smear = array('d', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
#chi2 = [5.40, 5.42, 5.11, 5.07, 4.84, 4.63, 4.23, 3.49, 3.42, 2.65, 1.84, 1.69]
sig_lam = array('d', [0.0206, 0.0217, 0.0233, 0.0244, 0.0256, 0.0276, 0.0305, 0.0335, 0.0366, 0.0428, 0.0500, 0.0566])
err_sig_lam = array('d', [0.0001, 0.0002, 0.0002, 0.0002, 0.0002, 0.0003, 0.0003, 0.0004, 0.0005, 0.0009, 0.0005, 0.0008])
sig_sig = array('d', [0.0246, 0.0254, 0.0281, 0.0291, 0.0302, 0.0332, 0.0368, 0.0379, 0.0420, 0.0497, 0.0534, 0.0583])
err_sig_sig = array('d', [0.0003, 0.0003, 0.0004, 0.0005, 0.0005, 0.0007, 0.0009, 0.0010, 0.0014, 0.0028, 0.0009, 0.0015])
gr = ROOT.TGraphErrors(len(p_smear), p_smear, sig_lam, err_p_smear, err_sig_lam)
#gr = ROOT.TGraph(len(p_smear), p_smear, sig_lam)
gr.SetTitle("#Lambda/#Sigma width in MM(eK^{+}K^{+}K^{-}) from MC (smeared))")
gr.GetXaxis().SetTitle("Gaussian smearing #Delta p/p(%)")
gr.GetYaxis().SetTitle("#sigma [GeV/c^{2}]")
gr.SetMarkerStyle(21)
gr.SetMarkerSize(0.8)
gr.SetLineColor(ROOT.kRed)
gr.SetLineStyle(7)
gr.SetMarkerColor(ROOT.kGreen)
#gr.GetYaxis().SetRangeUser(-0, 0.064)
gr.SetFillStyle(3004)

gr1 = ROOT.TGraphErrors(len(p_smear), p_smear, sig_sig, err_p_smear, err_sig_sig)
#gr1 = ROOT.TGraph(len(p_smear), p_smear, sig_sig)
gr1.SetTitle("#sigma(of #Sigma fit from MC)")
gr1.GetXaxis().SetTitle("Gaussian smearing #Delta p/p(%)")
gr1.GetYaxis().SetTitle("#sigma [GeV/c^{2}]")
gr1.SetMarkerStyle(21)
gr1.SetMarkerSize(0.8)
gr1.SetLineColor(ROOT.kGreen)
gr1.SetLineStyle(7)
gr1.SetMarkerColor(ROOT.kRed)
#gr1.GetYaxis().SetRangeUser(-0, 0.064)
gr1.SetFillStyle(3004)

%jsroot on
ROOT.gStyle.SetOptTitle(1)
c = ROOT.TCanvas()
c.SetTitle("MC smearing")
gr.Draw("AC*")
gr1.Draw("SAME")


t1 = ROOT.TLatex(1.1, 0.04, "#Sigma width from data")
t1.SetTextColor(ROOT.kBlue)
t1.Draw("SAME")

t2 = ROOT.TLatex(1.1, 0.03, "#Lambda width from data")
t2.SetTextColor(ROOT.kMagenta)
t2.Draw("SAME")


#l=ROOT.TLine(c.GetUxmin(),0.0275,c.GetUxmax(),0.0275) #0.0203+(3*0.0024)=0.0275 or 0.0227
#l=ROOT.TLine(c.GetUxmin(),0.0251,c.GetUxmax(),0.0251) #0.0203+(2*0.0024)
pt1 = ROOT.TPave(0,0.0179,1.4,0.0227, 0)
l=ROOT.TLine(c.GetUxmin(),0.0203,c.GetUxmax(),0.0203) #0.0203+(3*0.0024)=0.0275
l.SetLineColor(ROOT.kMagenta)
#l.SetTitle("#sigma(of #Lambda fit from data)")
l.Draw("SAME")
pt1.Draw("SAME")
pt1.SetFillStyle(3004)
pt1.SetFillColor(ROOT.kMagenta)#40#7#9


#l2=ROOT.TLine(c.GetUxmin(),0.0392,c.GetUxmax(),0.0392) #0.0206+(3*0.0062)=0.0392
#l2=ROOT.TLine(c.GetUxmin(),0.033,c.GetUxmax(),0.033) #0.0206+(2*0.0062)
pt2 = ROOT.TPave(0,0.0144,1.4,0.0268, 0)
l2=ROOT.TLine(c.GetUxmin(),0.0206,c.GetUxmax(),0.0206) #0.0206+(3*0.0062)=0.0392
l2.SetLineColor(ROOT.kBlue)
#l2.SetName("#sigma(of #Sigma fit from data) ")
l2.Draw("SAME")
pt2.Draw("SAME")
pt2.SetFillStyle(3004)
pt2.SetFillColor(ROOT.kBlue)#40#7#9

c.Draw()
#c.BuildLegend()

**MM(eK^{+}K^{+}K^{-}) Data Vs MC Fall2018-out-fde selecting Lambda/Sigma signal region**

In [25]:
#MM(ek+K+K-)
#MC files
#h1 = ROOT.RDataFrame("T","../simu_prod/xi_mc_fall2018out_fde/plot/rec_xi_mc_fall2018out_fde.root").Filter("1.0675<=mm_fkpskpkm && mm_fkpskpkm<=1.2481").Histo1D(("mc-rec", "mc-rec", 25, 0.9, 1.4), "mm_fkpskpkm")
#h1 = ROOT.RDataFrame("T","../simu_prod/xi_mc_fall2018out_fde/plot/rec_xi_mc_smeared0p1p_fall2018out_fde.root").Filter("1.0675<=mm_fkpskpkm && mm_fkpskpkm<=1.2481").Histo1D(("mc-rec", "mc-rec", 25, 0.9, 1.4), "mm_fkpskpkm")
#h1 = ROOT.RDataFrame("T","../simu_prod/xi_mc_fall2018out_fde/plot/rec_xi_mc_smeared0p2p_fall2018out_fde.root").Filter("1.0675<=mm_fkpskpkm && mm_fkpskpkm<=1.2481").Histo1D(("mc-rec", "mc-rec", 25, 0.9, 1.4), "mm_fkpskpkm")
#h1 = ROOT.RDataFrame("T","../simu_prod/xi_mc_fall2018out_fde/plot/rec_xi_mc_smeared0p3p_fall2018out_fde.root").Filter("1.0675<=mm_fkpskpkm && mm_fkpskpkm<=1.2481").Histo1D(("mc-rec", "mc-rec", 25, 0.9, 1.4), "mm_fkpskpkm")
#h1 = ROOT.RDataFrame("T","../simu_prod/xi_mc_fall2018out_fde/plot/rec_xi_mc_smeared0p35p_fall2018out_fde.root").Filter("1.0675<=mm_fkpskpkm && mm_fkpskpkm<=1.2481").Histo1D(("mc-rec", "mc-rec", 25, 0.9, 1.4), "mm_fkpskpkm")
#h1 = ROOT.RDataFrame("T","../simu_prod/xi_mc_fall2018out_fde/plot/rec_xi_mc_smeared0p4p_fall2018out_fde.root").Filter("1.0675<=mm_fkpskpkm && mm_fkpskpkm<=1.2481").Histo1D(("mc-rec", "mc-rec", 25, 0.9, 1.4), "mm_fkpskpkm")
#h1 = ROOT.RDataFrame("T","../simu_prod/xi_mc_fall2018out_fde/plot/rec_xi_mc_smeared0p5p_fall2018out_fde.root").Filter("1.0675<=mm_fkpskpkm && mm_fkpskpkm<=1.2481").Histo1D(("mc-rec", "mc-rec", 25, 0.9, 1.4), "mm_fkpskpkm")
#h1 = ROOT.RDataFrame("T","../simu_prod/xi_mc_fall2018out_fde/plot/rec_xi_mc_smeared0p6p_fall2018out_fde.root").Filter("1.0675<=mm_fkpskpkm && mm_fkpskpkm<=1.2481").Histo1D(("mc-rec", "mc-rec", 25, 0.9, 1.4), "mm_fkpskpkm")
#h1 = ROOT.RDataFrame("T","../simu_prod/xi_mc_fall2018out_fde/plot/rec_xi_mc_smeared0p7p_fall2018out_fde.root").Filter("1.0675<=mm_fkpskpkm && mm_fkpskpkm<=1.2481").Histo1D(("mc-rec", "mc-rec", 25, 0.9, 1.4), "mm_fkpskpkm")
#h1 = ROOT.RDataFrame("T","../simu_prod/xi_mc_fall2018out_fde/plot/rec_xi_mc_smeared0p8p_fall2018out_fde.root").Filter("1.0675<=mm_fkpskpkm && mm_fkpskpkm<=1.2481").Histo1D(("mc-rec", "mc-rec", 25, 0.9, 1.4), "mm_fkpskpkm")
#h1 = ROOT.RDataFrame("T","../simu_prod/xi_mc_fall2018out_fde/plot/rec_xi_mc_smeared1p_fall2018out_fde.root").Filter("1.0675<=mm_fkpskpkm && mm_fkpskpkm<=1.2481").Histo1D(("mc-rec", "mc-rec", 25, 0.9, 1.4), "mm_fkpskpkm")
#h1 = ROOT.RDataFrame("T","../simu_prod/xi_mc_fall2018out_fde/plot/rec_xi_mc_smeared1p2p_fall2018out_fde.root").Filter("1.0675<=mm_fkpskpkm && mm_fkpskpkm<=1.2481").Histo1D(("mc-rec", "mc-rec", 25, 0.9, 1.4), "mm_fkpskpkm")
h1 = ROOT.RDataFrame("T","../simu_prod/xi_mc_fall2018out_fde/plot/rec_xi_mc_smeared1p4p_fall2018out_fde.root").Filter("1.0675<=mm_fkpskpkm && mm_fkpskpkm<=1.2481").Histo1D(("mc-rec", "mc-rec", 25, 0.9, 1.4), "mm_fkpskpkm")
#h1 = ROOT.RDataFrame("T","../simu_prod/xi_mc_fall2018out_fde/plot/rec_xi_mc_smeared_fall2018out_fde.root").Filter("1.0675<=mm_fkpskpkm && mm_fkpskpkm<=1.2481").Histo1D(("mc-rec", "mc-rec", 25, 0.9, 1.4), "mm_fkpskpkm")
#h1 = ROOT.RDataFrame("T","../simu_prod/xi_mc_fall2018out_fde/plot/rec_xi_mc_smeared_gfun_fall2018out_fde.root").Filter("1.0675<=mm_fkpskpkm && mm_fkpskpkm<=1.2481").Histo1D(("mc-rec", "mc-rec", 25, 0.9, 1.4), "mm_fkpskpkm")

#DATA file
h2 = ROOT.RDataFrame("T","data_fall2018_out_fde.root").Filter("1.0675<=mm_fkpskpkm && mm_fkpskpkm<=1.2481").Histo1D(("data", "data", 25, 0.9, 1.4), "mm_fkpskpkm")
h1.Scale(1/h1.Integral())
h2.Scale(1/h2.Integral())
h1.GetXaxis().SetTitle("MM(eK^{+}K^{+}K^{-}) [GeV/c^{2}]")
#h1.GetYaxis().SetTitle("Counts/10 MeV/c^{2}")
h1.GetYaxis().SetTitle("Counts")
h1.SetMarkerStyle(25)
h1.SetMarkerSize(0.5)
h2.GetXaxis().SetTitle("MM(eK^{+}K^{+}K^{-}) [GeV/c^{2}]")
#h2.GetYaxis().SetTitle("Counts/10 MeV/c^{2}")
h2.GetYaxis().SetTitle("Counts")
h2.SetMarkerStyle(21)
h2.SetMarkerSize(0.5)
h1.GetXaxis().SetTitleSize(0.05)
h1.GetYaxis().SetTitleSize(0.05)
h1.GetXaxis().SetTitleOffset(0.9)
#h1.GetYaxis().SetTitleOffset(0.9)
h2.GetXaxis().SetTitleSize(0.05)
h2.GetYaxis().SetTitleSize(0.05)
h2.GetXaxis().SetTitleOffset(0.9)
#h.GetYaxis().SetTitleOffset(0.9)

#MM(eK+K+)
#MC files
#h_xi_mc = ROOT.RDataFrame("T","../simu_prod/xi_mc_fall2018out_fde/plot/rec_xi_mc_fall2018out_fde.root").Filter("1.0675<=mm_fkpskpkm && mm_fkpskpkm<=1.2481").Histo1D(("mc-im", "mc-im", 50, 1.5, 3.5), "mm_fkpskp")
#h_xi_mc = ROOT.RDataFrame("T","../simu_prod/xi_mc_fall2018out_fde/plot/rec_xi_mc_smeared0p1p_fall2018out_fde.root").Filter("1.0675<=mm_fkpskpkm && mm_fkpskpkm<=1.2481").Histo1D(("mc-im", "mc-im", 50, 1.5, 3.5), "mm_fkpskp")
#h_xi_mc = ROOT.RDataFrame("T","../simu_prod/xi_mc_fall2018out_fde/plot/rec_xi_mc_smeared0p2p_fall2018out_fde.root").Filter("1.0675<=mm_fkpskpkm && mm_fkpskpkm<=1.2481").Histo1D(("mc-im", "mc-im", 50, 1.5, 3.5), "mm_fkpskp")
#h_xi_mc = ROOT.RDataFrame("T","../simu_prod/xi_mc_fall2018out_fde/plot/rec_xi_mc_smeared0p3p_fall2018out_fde.root").Filter("1.0675<=mm_fkpskpkm && mm_fkpskpkm<=1.2481").Histo1D(("mc-im", "mc-im", 50, 1.5, 3.5), "mm_fkpskp")
#h_xi_mc = ROOT.RDataFrame("T","../simu_prod/xi_mc_fall2018out_fde/plot/rec_xi_mc_smeared0p35p_fall2018out_fde.root").Filter("1.0675<=mm_fkpskpkm && mm_fkpskpkm<=1.2481").Histo1D(("mc-im", "mc-im", 50, 1.5, 3.5), "mm_fkpskp")
#h_xi_mc = ROOT.RDataFrame("T","../simu_prod/xi_mc_fall2018out_fde/plot/rec_xi_mc_smeared0p4p_fall2018out_fde.root").Filter("1.0675<=mm_fkpskpkm && mm_fkpskpkm<=1.2481").Histo1D(("mc-im", "mc-im", 50, 1.5, 3.5), "mm_fkpskp")
#h_xi_mc = ROOT.RDataFrame("T","../simu_prod/xi_mc_fall2018out_fde/plot/rec_xi_mc_smeared0p5p_fall2018out_fde.root").Filter("1.0675<=mm_fkpskpkm && mm_fkpskpkm<=1.2481").Histo1D(("mc-im", "mc-im", 50, 1.5, 3.5), "mm_fkpskp")
#h_xi_mc = ROOT.RDataFrame("T","../simu_prod/xi_mc_fall2018out_fde/plot/rec_xi_mc_smeared0p6p_fall2018out_fde.root").Filter("1.0675<=mm_fkpskpkm && mm_fkpskpkm<=1.2481").Histo1D(("mc-im", "mc-im", 50, 1.5, 3.5), "mm_fkpskp")
#h_xi_mc = ROOT.RDataFrame("T","../simu_prod/xi_mc_fall2018out_fde/plot/rec_xi_mc_smeared0p7p_fall2018out_fde.root").Filter("1.0675<=mm_fkpskpkm && mm_fkpskpkm<=1.2481").Histo1D(("mc-im", "mc-im", 50, 1.5, 3.5), "mm_fkpskp")
#h_xi_mc = ROOT.RDataFrame("T","../simu_prod/xi_mc_fall2018out_fde/plot/rec_xi_mc_smeared0p8p_fall2018out_fde.root").Filter("1.0675<=mm_fkpskpkm && mm_fkpskpkm<=1.2481").Histo1D(("mc-im", "mc-im", 50, 1.5, 3.5), "mm_fkpskp")
#h_xi_mc = ROOT.RDataFrame("T","../simu_prod/xi_mc_fall2018out_fde/plot/rec_xi_mc_smeared1p_fall2018out_fde.root").Filter("1.0675<=mm_fkpskpkm && mm_fkpskpkm<=1.2481").Histo1D(("mc-im", "mc-im", 50, 1.5, 3.5), "mm_fkpskp")
#h_xi_mc = ROOT.RDataFrame("T","../simu_prod/xi_mc_fall2018out_fde/plot/rec_xi_mc_smeared1p2p_fall2018out_fde.root").Filter("1.0675<=mm_fkpskpkm && mm_fkpskpkm<=1.2481").Histo1D(("mc-im", "mc-im", 50, 1.5, 3.5), "mm_fkpskp")
h_xi_mc = ROOT.RDataFrame("T","../simu_prod/xi_mc_fall2018out_fde/plot/rec_xi_mc_smeared1p4p_fall2018out_fde.root").Filter("1.0675<=mm_fkpskpkm && mm_fkpskpkm<=1.2481").Histo1D(("mc-im", "mc-im", 50, 1.5, 3.5), "mm_fkpskp")
#h_xi_mc = ROOT.RDataFrame("T","../simu_prod/xi_mc_fall2018out_fde/plot/rec_xi_mc_smeared_fall2018out_fde.root").Filter("1.0675<=mm_fkpskpkm && mm_fkpskpkm<=1.2481").Histo1D(("mc-im", "mc-im", 50, 1.5, 3.5), "mm_fkpskp")
#h_xi_mc = ROOT.RDataFrame("T","../simu_prod/xi_mc_fall2018out_fde/plot/rec_xi_mc_smeared_gfun_fall2018out_fde.root").Filter("1.0675<=mm_fkpskpkm && mm_fkpskpkm<=1.2481").Histo1D(("mc-im", "mc-im", 50, 1.5, 3.5), "mm_fkpskp")

h_xi_mc.Scale(1/h_xi_mc.Integral())
#h_xi_mc.Scale(1/h_xi_mc.Integral(), "width")
h_xi_mc.GetXaxis().SetTitle("MM(eK^{+}K^{+}) [GeV/c^{2}]")
#h_xi_mc.GetYaxis().SetTitle("Counts/40 MeV/c^{2}")
h_xi_mc.GetYaxis().SetTitle("Counts")
h_xi_mc.SetMarkerStyle(25)
h_xi_mc.SetMarkerSize(0.5)
h_xi_mc.GetXaxis().SetTitleSize(0.05)
h_xi_mc.GetYaxis().SetTitleSize(0.05)
h_xi_mc.GetXaxis().SetTitleOffset(0.9)
#h_xi_mc.GetYaxis().SetTitleOffset(0.9)

#DATA
h_xi_data = ROOT.RDataFrame("T","data_fall2018_out_fde.root").Filter("1.0675<=mm_fkpskpkm && mm_fkpskpkm<=1.2481").Histo1D(("data-im", "data-im", 50, 1.5, 3.5), "mm_fkpskp")
h_xi_data.Scale(1/h_xi_data.Integral())
#h_xi_data.Scale(1/h_xi_data.Integral(), "width")
h_xi_data.SetMarkerStyle(21)
h_xi_data.SetMarkerSize(0.5)
h_xi_data.GetXaxis().SetTitle("MM(eK^{+}K^{+}) [GeV/c^{2}]")
h_xi_data.GetYaxis().SetTitle("Counts/40 MeV/c^{2}")
h_xi_data.GetYaxis().SetTitle("Counts")
h_xi_data.GetXaxis().SetTitleSize(0.05)
h_xi_data.GetYaxis().SetTitleSize(0.05)
h_xi_data.GetXaxis().SetTitleOffset(0.9)
#h_xi_data.GetYaxis().SetTitleOffset(0.9)

print(f"Integral of 3Kaon MM from data and mc::, {h2.Integral()} and {h1.Integral()}")
gof_test = 0.0
for i in range(h1.GetNbinsX()):
    #print(f"bin no. {i+1} has normalized data, {h2.GetBinContent(i+1)} and normalized MC, {h1.GetBinContent(i+1)}")
    data = h2.GetBinContent(i+1)
    err = h2.GetBinError(i+1)
    mc = h1.GetBinContent(i+1)
    if data>0 and mc>0:
        gof_test += math.pow(data-mc, 2)/math.pow(err, 2)
        #gof_test = math.pow(mc-data, 2)*10000
            
#print(f"Sum of squares (*10000):: {gof_test:.2f}")
print(f"Chi squares:: {gof_test:.2f}")

%jsroot on
c = ROOT.TCanvas()
ROOT.gStyle.SetOptStat(0000)
c.Divide(1,2)
c.cd(1)
h2.Draw("E")#data
#h2.Draw("HISTE")
h2.SetTitle("MM(eK^{+}K^{+}K^{-})")
#h1.GetYaxis().SetRangeUser(0, 0.3)
h2.SetLineColor(ROOT.kRed)
h1.Draw("SAMEHIST")
h1.SetLineColor(ROOT.kBlue)#mc

t1 = ROOT.TLatex(.94, 0.15, f"Chi square = {gof_test:0.2f}")
t1.Draw()
legend1 = ROOT.TLegend(0.9,0.8,0.7,0.9)#legend = ROOT.TLegend()
legend1.SetHeader("fall2018-out-fde","C") # option "C" allows to center the header
legend1.AddEntry("data","data","lpe")#lpe
#legend1.AddEntry("mc","mc-psmear-non","f")
#legend1.AddEntry("mc","mc-psmeard-0.1%","f")
#legend1.AddEntry("mc","mc-psmeard-0.2%","f")
#legend1.AddEntry("mc","mc-psmeard-0.3%","f")
#legend1.AddEntry("mc","mc-psmeard-0.35%","f")
#legend1.AddEntry("mc","mc-psmeard-0.4%","f")
#legend1.AddEntry("mc","mc-psmeard-0.5%","f")
#legend1.AddEntry("mc","mc-psmeard-0.6%","f")
#legend1.AddEntry("mc","mc-psmeard-0.7%","f")
#legend1.AddEntry("mc","mc-psmeard-0.8%","f")
#legend1.AddEntry("mc","mc-psmeard-1%","f")
#legend1.AddEntry("mc","mc-psmeard-1.2%","f")
legend1.AddEntry("mc","mc-psmeard-1.4%","f")
legend1.SetTextFont(72)
legend1.SetTextSize(0.8)
legend1.Draw()

c.cd(2)
h_xi_mc.Draw("HIST")
h_xi_mc.SetTitle("MM(eK^{+}K^{+})")
h_xi_mc.SetLineColor(ROOT.kBlue)
h_xi_mc.GetYaxis().SetRangeUser(0, 0.12)
h_xi_data.Draw("SAME")
#h_xi_data.Draw("SAMEE")
h_xi_data.SetLineColor(ROOT.kRed)
#h_im1.Scale(1/h_im1.Integral())
#h_im1.Draw("SAME")
legend = ROOT.TLegend(0.9,0.8,0.7,0.9)#legend = ROOT.TLegend()
legend.SetHeader("MC Vs data","C") # option "C" allows to center the header
legend.AddEntry("mc-im","mc","f")
legend.AddEntry("data-im","data-fall2018-out-fde","lpe")#lpe
#legend.Draw()

c.Draw()

Integral of 3Kaon MM from data and mc::, 1.0 and 1.0000000000000002
Chi squares:: 56.20


**Chi2 Minimization**

In [27]:
p_smear = array('d', [0, 0.1, 0.2, 0.3, 0.35, 0.4, 0.5, 0.6, 0.7, 0.8, 1.0, 1.2, 1.4])
chi2 = array('d', [28.67, 29.75, 26.44, 23.28, 21.27, 21.56, 23.63, 26.36, 29.84, 34.28, 45.27, 52.26, 56.26])
gr = ROOT.TGraph(len(p_smear), p_smear, chi2)
gr.SetTitle("MM(eK^{+}K^{+}K^{-}) Data(Fall2018outfde)/MC(smeared) matching")
gr.GetXaxis().SetTitle("Gaussian smearing factor #Delta p/p(%)")
gr.GetYaxis().SetTitle("Chi square")
gr.SetMarkerStyle(21)
gr.SetMarkerSize(0.8)
gr.SetLineColor(ROOT.kRed)
gr.SetMarkerColor(ROOT.kGreen)

%jsroot on
ROOT.gStyle.SetOptTitle(1)
c = ROOT.TCanvas()
gr.Draw("AC*")

c.Draw()
#c.BuildLegend()